In [51]:
import pandas as pd
import numpy as np
# import jqdatasdk as jq
import statsmodels.api as sm
import time,datetime
import os
from scipy import stats

In [2]:
# jq.auth('13918852005','960312Lsc')
# jq.auth('15821912507','912507')

auth success （JQData现有流量增加活动，详情请咨询JQData管理员，微信号：JQData01）


In [ ]:
# 先分组，5因子分组：市值2组，其余3组

In [7]:
# 生成分组标识
bin_df = pd.read_csv('E:/Stock_Data/bin_data.csv',index_col=0)
for col in ['B_M', 'OP','pe_ratio']:
    if isinstance(bin_df[col][0],float):
        bin_df[col] = bin_df[col].apply(lambda x : np.nan if x<0 else x)
bin_df = bin_df.dropna(how='any',axis=0)

## 分组组数
bin_num = 3

for i in range(8):
    
    year = str(2010+i)+'-12-31'
    temp_df_0 = bin_df[bin_df.date==year]
    
    # 计算当年总行数
    stock_num = len(temp_df_0)
    
    for bin_kind in ['B_M','OP','Inv']:
        
        # 生成分组标识
        bin_lst = []
        for i in range(bin_num):
            bin_lst += [bin_kind+str(i+1)]*int(stock_num/bin_num)
        if len(bin_lst) != stock_num:
            bin_lst += [bin_kind+str(i+1)]*(stock_num-len(bin_lst))
        
        temp_df_0 = temp_df_0.sort_values(bin_kind)
        temp_df_0[bin_kind] = bin_lst
            
    for bin_kind in ['Size']:
        
        # Size只分两组
        bin_num_size = 2
        # 生成分组标识
        bin_lst = []
        for i in range(bin_num_size):
            bin_lst += [bin_kind+str(i+1)]*int(stock_num/bin_num_size)
        if len(bin_lst) != stock_num:
            bin_lst += [bin_kind+str(i+1)]*(stock_num-len(bin_lst))
        
        temp_df_0 = temp_df_0.sort_values(bin_kind)
        temp_df_0[bin_kind] = bin_lst    
    
    temp_df_1 = bin_df[bin_df.date==year]
    temp_df_1 = temp_df_1.sort_values('Size') # 和df_0对齐，最后一次是按照Size排序的
    temp_df_0['S']=temp_df_1['Size']
    temp_df_0.to_csv('E:/Stock_Data/factor_portfolio/bin_data'+year+'.csv',index=True)

In [8]:
# 按要求生成投资组合

# 保存在字典：除Size以外的另一个组别 -> 年份 -> bin2 -> 组合，stock code list

portfolio={}

for bin2 in ['B_M','OP','Inv','Size']:
    portfolio[bin2] = {}
    
    for i in range(8):
        year = str(2010+i)+'-12-31'
        portfolio[bin2][str(2011+i)]={}
        
        temp_df_0 = pd.read_csv('E:/Stock_Data/factor_portfolio/bin_data'+year+'.csv',index_col=0)
        bin2_lst = list(set(temp_df_0[bin2]))
        
        for bin2_ in bin2_lst:
            temp_df_1 = temp_df_0[temp_df_0[bin2]==bin2_]
            list_code_value = [list(temp_df_1[['code','S']].iloc[i,:]) for i in range(len(temp_df_1))]
            portfolio[bin2][str(2011+i)][bin2_] = list_code_value
            
# 保存字典
f = open('E:/Stock_Data/factor_portfolio/portfolio.txt','w')
f.write(str(portfolio))
f.close()

In [9]:
# 读取字典
f = open('E:/Stock_Data/factor_portfolio/portfolio.txt','r')
a = f.read()
portfolio = eval(a)
f.close()

In [18]:
stock_download_return_temp = os.listdir('E:/Stock_Data/stock_return_data/')
stock_download_return = [x[:-4] for x in stock_download_return_temp]

In [19]:
stock_download_return

['000001.XSHE',
 '000002.XSHE',
 '000004.XSHE',
 '000005.XSHE',
 '000006.XSHE',
 '000008.XSHE',
 '000009.XSHE',
 '000011.XSHE',
 '000012.XSHE',
 '000014.XSHE',
 '000016.XSHE',
 '000017.XSHE',
 '000018.XSHE',
 '000019.XSHE',
 '000020.XSHE',
 '000021.XSHE',
 '000022.XSHE',
 '000023.XSHE',
 '000024.XSHE',
 '000025.XSHE',
 '000026.XSHE',
 '000027.XSHE',
 '000028.XSHE',
 '000030.XSHE',
 '000031.XSHE',
 '000032.XSHE',
 '000034.XSHE',
 '000036.XSHE',
 '000037.XSHE',
 '000039.XSHE',
 '000040.XSHE',
 '000042.XSHE',
 '000043.XSHE',
 '000045.XSHE',
 '000046.XSHE',
 '000048.XSHE',
 '000049.XSHE',
 '000050.XSHE',
 '000055.XSHE',
 '000056.XSHE',
 '000058.XSHE',
 '000059.XSHE',
 '000060.XSHE',
 '000061.XSHE',
 '000062.XSHE',
 '000063.XSHE',
 '000065.XSHE',
 '000066.XSHE',
 '000069.XSHE',
 '000070.XSHE',
 '000078.XSHE',
 '000088.XSHE',
 '000089.XSHE',
 '000090.XSHE',
 '000096.XSHE',
 '000099.XSHE',
 '000100.XSHE',
 '000150.XSHE',
 '000151.XSHE',
 '000153.XSHE',
 '000157.XSHE',
 '000158.XSHE',
 '000159

In [20]:
# 对于某一投资组合 bin2 计算所有收益率序列
address = 'E:/Stock_Data/stock_return_data/'
bin2s = ['B_M','OP','Inv','Size']
return_inf_used_stock = list(pd.read_csv('E:/Stock_Data/return_inf_used_stock.csv').iloc[:,0])
stock_download_return_temp = os.listdir('E:/Stock_Data/stock_return_data/')
stock_download_return = [x[:-4] for x in stock_download_return_temp]

## 第一层，分组依据
for bin2 in bin2s:
    portfolio_0 = portfolio[bin2]
    
    ## 第二层，不同年份
    for i in range(8):
        year = str(2011+i)
        portfolio_1 = portfolio_0[year]
        cnt_bin = 0
        
        ## 第三层，不同小组
        for bin1_bin2_type in portfolio_1.keys():
            cnt = 0
            tol_weight = 0
            portfolio_2 = portfolio_1[bin1_bin2_type]
            for code_weight in portfolio_2:
                
                code = code_weight[0]
                weight = code_weight[1]
                
                # 股票由于停牌时间过长被剔除
                if code not in stock_download_return:
                    continue
                
                file_name = address+code+'.csv'
                r = pd.read_csv(file_name,index_col=0)
                
                # 读取到的序列过短，剔除
                if len(r)<1943:
                    continue
                
                if cnt == 0:
                    r_sigma = r*weight
                else:
                    r_sigma = r_sigma+r*weight
                
                tol_weight += weight       
                cnt+=1  
            
            ### 求加权平均数，得到一小组的收益率序列
            r_mean = r_sigma/tol_weight
            r_mean.columns = [bin1_bin2_type]
            if cnt_bin == 0:
                r_save = r_mean
            else:
                r_save = pd.concat([r_save,r_mean],axis=1)
            cnt_bin += 1
        ### 删选当年的数据
        r_save_0 = r_save[r_save.index< str(int(year)+1)+'-01-01']
        r_save_1 = r_save_0[r_save_0.index> str(int(year)-1)+'-12-31']
        ### 每一年拼接
        if i == 0:
            r_save_year = r_save_1
        else:
            r_save_year = pd.concat([r_save_year,r_save_1],axis=0)
    
    ### 完成一个大组的每一年计算后，保存
    r_save_year.to_csv('E:/Stock_Data/factor_portfolio/'+bin2+'.csv',index=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [ ]:
# 因子计算

In [3]:
# SMB small minus big - Size
Size = pd.read_csv('E:/Stock_Data/factor_portfolio/Size.csv',index_col=0).iloc[:1943,:]
SMB = Size['Size1'] - Size['Size2']
SMB = pd.DataFrame(SMB,columns=['SMB'])

In [4]:
# HML high minus low - B_M
B_M = pd.read_csv('E:/Stock_Data/factor_portfolio/B_M.csv',index_col=0).iloc[:1943,:]
HML = B_M['B_M3'] - B_M['B_M1']
HML = pd.DataFrame(HML,columns=['HML'])

In [5]:
# RMW robust minus weak - OP
OP = pd.read_csv('E:/Stock_Data/factor_portfolio/OP.csv',index_col=0).iloc[:1943,:]
RMW = OP['OP3'] - OP['OP1']
RMW.columns = ['RMW']
RMW = pd.DataFrame(RMW,columns=['RMW'])

In [6]:
# CMA conservative minus aggressive - Inv
Inv = pd.read_csv('E:/Stock_Data/factor_portfolio/Inv.csv',index_col=0).iloc[:1943,:]
CMA = Inv['Inv1'] - Inv['Inv3']
CMA = pd.DataFrame(CMA,columns=['CMA'])

In [89]:
# market risk，读取投资组合，选取一个标准的全部样本（全样本），计算加权均值，即 Rm

# 读取字典
f = open('E:/Stock_Data/factor_portfolio/portfolio.txt','r')
a = f.read()
portfolio = eval(a)
f.close()
portfolio_0 = portfolio['Size']
## 第二层，不同年份
for i in range(8):
    year = str(2011+i)
    portfolio_1 = portfolio_0[year]
    tol_weight = 0 # 把总权重放置年份后，对每小组不再单独求和，即Size1+Size2=全股票
    cnt = 0 # 进对每一年清零计数

    ## 第三层，不同小组
    for bin1_bin2_type in portfolio_1.keys():


        portfolio_2 = portfolio_1[bin1_bin2_type]
        for code_weight in portfolio_2:

            code = code_weight[0]
            weight = code_weight[1]

            # 股票由于停牌时间过长被剔除
            if code not in stock_download_return:
                continue

            file_name = address+code+'.csv'
            r = pd.read_csv(file_name,index_col=0)

            # 读取到的序列过短，剔除
            if len(r)<1943:
                continue

            if cnt == 0:
                r_sigma = r*weight
            else:
                r_sigma = r_sigma+r*weight

            tol_weight += weight       
            cnt+=1  

    ### 求加权平均数，得到一年的收益率序列
    r_mean = r_sigma/tol_weight
    r_save = r_mean

    ### 删选当年的数据
    r_save_0 = r_save[r_save.index< str(int(year)+1)+'-01-01']
    r_save_1 = r_save_0[r_save_0.index> str(int(year)-1)+'-12-31']
    ### 每一年拼接
    if i == 0:
        r_save_year = r_save_1
    else:
        r_save_year = pd.concat([r_save_year,r_save_1],axis=0)

### 完成一个大组的每一年计算后，保存
r_save_year.columns = ['return']
r_save_year.to_csv('E:/Stock_Data/factor_portfolio/MarketRisk.csv',index=True)

In [9]:
# MKTF
MKT = pd.read_csv('E:/Stock_Data/factor_portfolio/MarketRisk.csv',index_col=0).iloc[:1943,:]
MKTF = MKT - rf
MKTF.columns = ['MKTF']

In [24]:
# rf
import re
rf_temp_0 = pd.read_csv('E:/Stock_Data/t_bill_one_year.csv')[['日期','收盘']]
rf_temp_0.index = rf_temp_0['日期'].apply(lambda x: x[:4]+'-'+re.findall('(.*?)年(.*?)月(.*?)日',x)[0][1].zfill(2)+'-'+re.findall('(.*?)年(.*?)月(.*?)日',x)[0][2].zfill(2))
rf_temp_1 = pd.merge(left = MKT,right = rf_temp_0,left_index = True,right_index=True,how='left')[['收盘']]
rf_temp_2 = rf_temp_1.fillna(rf_temp_1.mean())
rf_temp_2.columns = ['return']
rf = rf_temp_2/252/100
rf.to_csv('E:/Stock_Data/rf.csv')

NameError: name 'MKT' is not defined

In [7]:
# 引入日历效应哑变量 - 星期
sample_df = pd.read_csv('E:/Stock_Data/stock_return_data/001896.XSHE.csv',index_col=0)
trade_day_serises = list(sample_df.index)

trade_day_df = pd.DataFrame(trade_day_serises)
trade_day_df.columns = ['weekday']
trade_day_df['weekday'] = trade_day_df['weekday'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d").weekday()+1)

dummy_df = pd.get_dummies(trade_day_df['weekday'])
dummy_df = dummy_df[[1,2,3,4,5]]
dummy_df.index = sample_df.index

X = dummy_df.iloc[:1943,:]

In [15]:
# 检验各因子的周四效应
X = dummy_df.iloc[:1943,:]
for factor in [MKTF,SMB,HML,CMA,RMW]:
    y = factor

    est = sm.OLS(y,X)
    result = est.fit()

    print(result.params)
    print(result.pvalues)

1   -0.000484
2    0.000467
3    0.000099
4   -0.001767
5    0.000647
dtype: float64
1    0.491212
2    0.498646
3    0.884824
4    0.010359
5    0.349133
dtype: float64
1   -0.000056
2    0.000641
3    0.000765
4   -0.000763
5   -0.000571
dtype: float64
1    0.924734
2    0.267614
3    0.182846
4    0.185843
5    0.324120
dtype: float64
1    0.000911
2   -0.000229
3   -0.000039
4    0.001197
5    0.001046
dtype: float64
1    0.123331
2    0.692712
3    0.946678
4    0.038811
5    0.072119
dtype: float64
1    0.000593
2   -0.000278
3    0.000253
4    0.000077
5   -0.000220
dtype: float64
1    0.037010
2    0.320136
3    0.360773
4    0.783021
5    0.431464
dtype: float64
1   -0.000327
2    0.000082
3   -0.000290
4    0.001145
5    0.000890
dtype: float64
1    0.547852
2    0.877428
3    0.584072
4    0.031478
5    0.095713
dtype: float64


In [ ]:
# 获得5因子中具有公司特征的4个因子分组的投资组合收益率数据，观察负周四效应

In [4]:
# 生成分组标识
bin_df = pd.read_csv('E:/Stock_Data/bin_data.csv',index_col=0)
for col in ['B_M', 'OP','pe_ratio']:
    if isinstance(bin_df[col][0],float):
        bin_df[col] = bin_df[col].apply(lambda x : np.nan if x<0 else x)
bin_df = bin_df.dropna(how='any',axis=0)

## 分组组数
bin_num = 8

for i in range(8):
    
    year = str(2010+i)+'-12-31'
    temp_df_0 = bin_df[bin_df.date==year]
    
    # 计算当年总行数
    stock_num = len(temp_df_0)
    
    for bin_kind in ['B_M','OP','Inv','Size']:
        
        # 生成分组标识
        bin_lst = []
        for i in range(bin_num):
            bin_lst += [bin_kind+str(i+1)]*int(stock_num/bin_num)
        if len(bin_lst) != stock_num:
            bin_lst += [bin_kind+str(i+1)]*(stock_num-len(bin_lst))
        
        temp_df_0 = temp_df_0.sort_values(bin_kind)
        temp_df_0[bin_kind] = bin_lst
    
    temp_df_1 = bin_df[bin_df.date==year]
    temp_df_1 = temp_df_1.sort_values('Size') # 和df_0对齐，最后一次是按照Size排序的
    temp_df_0['S']=temp_df_1['Size']
    temp_df_0.to_csv('E:/Stock_Data/factor_portfolio/5factor'+year+'.csv',index=True)

In [2]:
# 按要求生成投资组合

# 保存在字典：除B_M以外的另一个组别 -> 年份 -> bin1 与 bin2 单独！ -> 组合，stock code list

portfolio={}

for bin2 in ['B_M','OP','Inv','margin','Size']:
    portfolio[bin2] = {}
    
    for i in range(8):
        year = str(2010+i)+'-12-31'
        portfolio[bin2][str(2011+i)]={}
        
        temp_df_0 = pd.read_csv('E:/Stock_Data/factor_portfolio/5factor'+year+'.csv',index_col=0)
        bin2_lst = list(set(temp_df_0[bin2]))
        
        for bin2_ in bin2_lst:  
            temp_df_2 = temp_df_0[temp_df_0[bin2]==bin2_]
            list_code_value = [list(temp_df_2[['code','S']].iloc[i,:]) for i in range(len(temp_df_2))]
            portfolio[bin2][str(2011+i)][bin2_] = list_code_value
# 保存字典
f = open('E:/Stock_Data/factor_portfolio/5factor_portfolio.txt','w')
f.write(str(portfolio))
f.close()

In [3]:
# 读取字典
f = open('E:/Stock_Data/factor_portfolio/5factor_portfolio.txt','r')
a = f.read()
portfolio = eval(a)
f.close()

In [4]:
# 对于某一投资组合 bin2 计算所有收益率序列 - 市值加权平均
address = 'E:/Stock_Data/stock_return_data/'

bin2s = ['B_M','OP','Inv','margin','Size']
stock_download_return_temp = os.listdir('E:/Stock_Data/stock_return_data/')
stock_download_return = [x[:-4] for x in stock_download_return_temp]

## 第一层，分组依据
for bin2 in bin2s:
    portfolio_0 = portfolio[bin2]
    
    ## 第二层，不同年份
    for i in range(8):
        year = str(2011+i)
        portfolio_1 = portfolio_0[year]
        cnt_bin = 0
        
        ## 第三层，不同小组
        for bin1_bin2_type in portfolio_1.keys():
            cnt = 0
            tol_weight = 0
            portfolio_2 = portfolio_1[bin1_bin2_type]
            for code_weight in portfolio_2:
                
                code = code_weight[0]
                weight = code_weight[1]
                
                # 股票由于停牌时间过长被剔除
                if code not in stock_download_return:
                    continue
                
                file_name = address+code+'.csv'
                r = pd.read_csv(file_name,index_col=0)
                
                # 读取到的序列过短，剔除
                if len(r)<1943:
                    continue
                
                if cnt == 0:
                    r_sigma = r*weight
                else:
                    r_sigma = r_sigma+r*weight
                
                tol_weight += weight       
                cnt+=1  
            
            ### 求加权平均数，得到一小组的收益率序列
            r_mean = r_sigma/tol_weight
            r_mean.columns = [bin1_bin2_type]
            if cnt_bin == 0:
                r_save = r_mean
            else:
                r_save = pd.concat([r_save,r_mean],axis=1)
            cnt_bin += 1
        ### 删选当年的数据
        r_save_0 = r_save[r_save.index< str(int(year)+1)+'-01-01']
        r_save_1 = r_save_0[r_save_0.index> str(int(year)-1)+'-12-31']
        ### 每一年拼接
        if i == 0:
            r_save_year = r_save_1
        else:
            r_save_year = pd.concat([r_save_year,r_save_1],axis=0)
    
    ### 完成一个大组的每一年计算后，保存
    r_save_year.to_csv('E:/Stock_Data/factor_portfolio/singlebin_'+bin2+'.csv',index=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [ ]:
bin2s = ['B_M','OP','Inv','Size']

for bin2 in bin2s:
    return_df = pd.read_csv('E:/Stock_Data/factor_portfolio/singlebin_'+bin2+'.csv',index_col=0)
    return_df = return_df.dropna(how="any",axis=0)
    bin2_list = [bin2+str(i+1) for i in range(8)]
    bin1_bin2_save_df = pd.DataFrame(index=[''],columns=bin2_list)
    
    col = 0
    for bin2_ in bin2_list:
        bin1_bin2_ = bin2_
        # y减去无风险收益率
        y_temp = return_df.iloc[:1943,:]
        y = y_temp[bin1_bin2_]
        X = dummy_df.iloc[:1943,:]

#             # 用Garch
#             reg = arch_model(y, x=X, mean='HARX', lags=0, vol='Garch', p=1, o=0, q=1, power=1.0, dist='ged', hold_back=None)
#             result = reg.fit()

        # 用OLS
#             X = sm.add_constant(X)
        est = sm.OLS(y,X)
        result = est.fit()           

        calandar_effect = list(result.params)[3]

        # 测试 - 结果矩阵形式保存
        bin1_bin2_save_df.iloc[0,col] = calandar_effect

        col += 1
    # 保存
    bin1_bin2_save_df.to_csv('E:/Stock_Data/factor_portfolio/'+ bin2 +'8bin_result.csv')

In [15]:
# 对于某一投资组合 bin2 计算所有收益率序列 - 算术&加权 平均
address = 'E:/Stock_Data/stock_return_data/'

bin2s = ['margin']
stock_download_return_temp = os.listdir('E:/Stock_Data/stock_return_data/')
stock_download_return = [x[:-4] for x in stock_download_return_temp]

## 第一层，分组依据
for bin2 in bin2s:
    portfolio_0 = portfolio[bin2]
    
    ## 第二层，不同年份
    for i in range(8):
        year = str(2011+i)
        portfolio_1 = portfolio_0[year]
        cnt_bin = 0
        
        ## 第三层，不同小组
        for bin1_bin2_type in portfolio_1.keys():
            cnt = 0
            tol_weight = 0
            portfolio_2 = portfolio_1[bin1_bin2_type]
            for code_weight in portfolio_2:
                
                code = code_weight[0]
                weight = code_weight[1]
                
                # 股票由于停牌时间过长被剔除
                if code not in stock_download_return:
                    continue
                
                file_name = address+code+'.csv'
                r = pd.read_csv(file_name,index_col=0)
                
                # 读取到的序列过短，剔除
                if len(r)<1943:
                    continue
                
                if cnt == 0:
                    r_sigma = r
                    r_sigma_w = r*weight
                else:
                    r_sigma = r_sigma+r
                    r_sigma_w = r_sigma_w+r*weight
                
                tol_weight += weight       
                cnt+=1  
            
            ### 求加权平均数，得到一小组的收益率序列
            r_mean = r_sigma/cnt
            r_mean_w = r_sigma_w/tol_weight
            
            r_mean.columns = [bin1_bin2_type]
            r_mean_w.columns = [bin1_bin2_type]
            
            if cnt_bin == 0:
                r_save = r_mean
                r_save_w = r_mean_w
            else:
                r_save = pd.concat([r_save,r_mean],axis=1)
                r_save_w = pd.concat([r_save_w,r_mean_w],axis=1)
            cnt_bin += 1
        ### 删选当年的数据
        r_save_0 = r_save[r_save.index< str(int(year)+1)+'-01-01']
        r_save_1 = r_save_0[r_save_0.index> str(int(year)-1)+'-12-31']
        r_save_0_w = r_save_w[r_save_w.index< str(int(year)+1)+'-01-01']
        r_save_1_w = r_save_0_w[r_save_0_w.index> str(int(year)-1)+'-12-31']
        ### 每一年拼接
        if i == 0:
            r_save_year = r_save_1
            r_save_year_w = r_save_1_w
        else:
            r_save_year = pd.concat([r_save_year,r_save_1],axis=0)
            r_save_year_w = pd.concat([r_save_year_w,r_save_1_w],axis=0)
    
    ### 完成一个大组的每一年计算后，保存
    r_save_year.to_csv('E:/Stock_Data/factor_portfolio/singlebin_'+bin2+'mean.csv',index=True)
    r_save_year_w.to_csv('E:/Stock_Data/factor_portfolio/singlebin_'+bin2+'weight.csv',index=True)

In [16]:
bin2s = ['margin']

for how in ['mean','weight']:
    for bin2 in bin2s:
        return_df = pd.read_csv('E:/Stock_Data/factor_portfolio/singlebin_'+bin2+how+'.csv',index_col=0)
        return_df = return_df.dropna(how="any",axis=0)
        bin2_list = ['margin','Nomargin']
        bin1_bin2_save_df = pd.DataFrame(index=['calandar_effect','Pvalue'],columns=bin2_list)

        col = 0
        for bin2_ in bin2_list:
            bin1_bin2_ = bin2_
            # y减去无风险收益率
            y_temp = return_df.iloc[:1943,:]
            y = y_temp[bin1_bin2_]
            X = dummy_df.iloc[:1943,:]

            # 用OLS
    #             X = sm.add_constant(X)
            est = sm.OLS(y,X)
            result = est.fit()           

            calandar_effect = list(result.params)[3]
            calandar_effect_sig = list(result.pvalues)[3]

            # 测试 - 结果矩阵形式保存
            bin1_bin2_save_df.iloc[0,col] = calandar_effect
            bin1_bin2_save_df.iloc[1,col] = calandar_effect_sig

            col += 1
        # 保存
        bin1_bin2_save_df.to_csv('E:/Stock_Data/factor_portfolio/'+ bin2 +how+'2bin_result.csv')

In [46]:
# 融资融券与非融资融券组的差别
r_margin_temp=pd.read_csv('E:/Stock_Data/factor_portfolio/singlebin_marginweight.csv',index_col=0)
r_margin=r_margin_temp.iloc[:1943,]

In [57]:
# 指数角度检验 - 周历效应 - 分时段
rf = pd.read_csv('E:/Stock_Data/rf.csv',index_col=0)
dummy_df=dummy_df.iloc[:1943,]
for time in range(5):
    t_test_lst=[]
#     start = str(2011+time)+'-01-04'
#     end = '2018-12-28'
    
    if time ==0:
        start = '2011-01-04'
        end = '2018-12-31'
    if time ==1:
        start = '2012-01-01'
        end = '2018-12-31'
    if time ==2:
        start = '2011-01-01'
        end = '2017-12-31'
    if time ==3:
        start = '2013-01-01'
        end = '2018-12-31'
    if time ==4:
        start = '2011-01-01'
        end = '2016-12-31'
#     if time ==3:
#         start = '2014-01-01'
#         end = '2015-12-31'
#     if time ==4:
#         start = '2015-01-01'
#         end = '2016-12-31'
    
    index_save_dct = {1:0,2:0,3:0,4:0,5:0}
    index_p_save_dct = {1:0,2:0,3:0,4:0,5:0}
    for index in ['margin','Nomargin']:
        
        index_df = r_margin[[index]]
        index_df.columns=['return']

        index_df = index_df[index_df.index>start]
        index_df = index_df[index_df.index<end]
        X = dummy_df[dummy_df.index>start]
        X = X[X.index<end]
        y = index_df
        
        tol_y_X = pd.concat([y,X],axis=1)
        t_test_lst.append(list(tol_y_X[tol_y_X[4]==1]['return']))

        # 用OLS
        # 是否使用超额收益率
#         temp_y = pd.merge(y,rf,left_index=True,right_index=True,how='left').fillna(rf.mean())
#         y = temp_y[['return_x']]
#         y.columns=['return']
#         rf = temp_y[['return_y']]
#         rf.columns=['return']
#         y = y - rf

        est = sm.OLS(y,X)
        result = est.fit()
        for i in range(5):
            index_save_dct[i+1] = '%.6f' %list(result.params)[i]
        for i in range(5):
            index_p_save_dct[i+1] = '%.6f' %list(result.pvalues)[i]
        f_p = '%.6f' %result.f_pvalue
        
        print('时段',time+1,'指数',index,'\n系数',index_save_dct,'\nP值',index_p_save_dct,'\nF统计量P值',f_p,'\n')
    print(stats.levene(t_test_lst[0],t_test_lst[1]))
    print(stats.ttest_rel(t_test_lst[0],t_test_lst[1]))
    print(stats.ttest_ind(t_test_lst[0],t_test_lst[1],equal_var = False))

时段 1 指数 margin 
系数 {1: '-0.000246', 2: '0.000574', 3: '0.000094', 4: '-0.001385', 5: '0.000900'} 
P值 {1: '0.722153', 2: '0.398582', 3: '0.889169', 4: '0.041162', 5: '0.186248'} 
F统计量P值 0.148441 

时段 1 指数 Nomargin 
系数 {1: '-0.000621', 2: '0.000852', 3: '0.000557', 4: '-0.002317', 5: '0.000317'} 
P值 {1: '0.480311', 2: '0.324214', 3: '0.515601', 4: '0.007216', 5: '0.714275'} 
F统计量P值 0.064645 

LeveneResult(statistic=8.69309655535009, pvalue=0.003289313680274249)
Ttest_relResult(statistic=1.97967816091891, pvalue=0.048440559705695475)
Ttest_indResult(statistic=0.8758219509787907, pvalue=0.38140475603593404)
时段 2 指数 margin 
系数 {1: '0.000067', 2: '0.000789', 3: '-0.000007', 4: '-0.001308', 5: '0.000896'} 
P值 {1: '0.929173', 2: '0.290259', 3: '0.992627', 4: '0.079646', 5: '0.231599'} 
F统计量P值 0.234255 

时段 2 指数 Nomargin 
系数 {1: '-0.000072', 2: '0.001288', 3: '0.000355', 4: '-0.002021', 5: '0.000277'} 
P值 {1: '0.940296', 2: '0.170311', 3: '0.703836', 4: '0.031579', 5: '0.769279'} 
F统计量P值 0.1508

In [66]:
# 对于某一投资组合 bin2 计算所有收益率序列 - 算术平均
address = 'E:/Stock_Data/stock_return_data/'

bin2s = ['B_M','OP','Inv','Size']
stock_download_return_temp = os.listdir('E:/Stock_Data/stock_return_data/')
stock_download_return = [x[:-4] for x in stock_download_return_temp]

## 第一层，分组依据
for bin2 in bin2s:
    portfolio_0 = portfolio[bin2]
    
    ## 第二层，不同年份
    for i in range(8):
        year = str(2011+i)
        portfolio_1 = portfolio_0[year]
        cnt_bin = 0
        
        ## 第三层，不同小组
        for bin1_bin2_type in portfolio_1.keys():
            cnt = 0
            tol_weight = 0
            portfolio_2 = portfolio_1[bin1_bin2_type]
            for code_weight in portfolio_2:
                
                code = code_weight[0]
                weight = code_weight[1]
                
                # 股票由于停牌时间过长被剔除
                if code not in stock_download_return:
                    continue
                
                file_name = address+code+'.csv'
                r = pd.read_csv(file_name,index_col=0)
                
                # 读取到的序列过短，剔除
                if len(r)<1943:
                    continue
                
                if cnt == 0:
                    r_sigma = r
                else:
                    r_sigma = r_sigma+r
                
                cnt+=1  
            
            ### 求加权平均数，得到一小组的收益率序列
            r_mean = r_sigma/cnt
            r_mean.columns = [bin1_bin2_type]
            if cnt_bin == 0:
                r_save = r_mean
            else:
                r_save = pd.concat([r_save,r_mean],axis=1)
            cnt_bin += 1
        ### 删选当年的数据
        r_save_0 = r_save[r_save.index< str(int(year)+1)+'-01-01']
        r_save_1 = r_save_0[r_save_0.index> str(int(year)-1)+'-12-31']
        ### 每一年拼接
        if i == 0:
            r_save_year = r_save_1
        else:
            r_save_year = pd.concat([r_save_year,r_save_1],axis=0)
    
    ### 完成一个大组的每一年计算后，保存
    r_save_year.to_csv('E:/Stock_Data/factor_portfolio/singlebin_mean_'+bin2+'.csv',index=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [67]:
bin2s = ['B_M','OP','Inv','Size']

for bin2 in bin2s:
    return_df = pd.read_csv('E:/Stock_Data/factor_portfolio/singlebin_mean_'+bin2+'.csv',index_col=0)
    return_df = return_df.dropna(how="any",axis=0)
    bin2_list = [bin2+str(i+1) for i in range(8)]
    bin1_bin2_save_df = pd.DataFrame(index=[''],columns=bin2_list)
    
    col = 0
    for bin2_ in bin2_list:
        bin1_bin2_ = bin2_
        # y减去无风险收益率
        y_temp = return_df.iloc[:1943,:]
        y = y_temp[bin1_bin2_]
        X = dummy_df.iloc[:1943,:]

#             # 用Garch
#             reg = arch_model(y, x=X, mean='HARX', lags=0, vol='Garch', p=1, o=0, q=1, power=1.0, dist='ged', hold_back=None)
#             result = reg.fit()

        # 用OLS
#             X = sm.add_constant(X)
        est = sm.OLS(y,X)
        result = est.fit()           

        calandar_effect = list(result.params)[3]

        # 测试 - 结果矩阵形式保存
        bin1_bin2_save_df.iloc[0,col] = calandar_effect

        col += 1
    # 保存
    bin1_bin2_save_df.to_csv('E:/Stock_Data/factor_portfolio/'+ bin2 +'8bin_result_mean.csv')

In [ ]:
#################################################################################################

In [17]:
# 计算得到年份平均的公司特征指标，对负周四效应回归
bin_data_df = pd.read_csv('E:/Stock_Data/bin_data.csv')
for col in ['B_M', 'OP','pe_ratio']:
    if isinstance(bin_data_df[col][0],float):
        bin_data_df[col] = bin_data_df[col].apply(lambda x : np.nan if x<0 else x)
bin_data_df = bin_data_df.dropna(how='any',axis=0)

# 剔除异常值
bin_data_df_mean = bin_data_df.groupby('code').mean()
bin_data_df_mean = bin_data_df_mean.sort_values('OP').iloc[int(len(bin_data_df_mean)*0.01):int(len(bin_data_df_mean)*0.99),]
bin_data_df_mean = bin_data_df_mean.sort_values('Inv').iloc[int(len(bin_data_df_mean)*0.01):int(len(bin_data_df_mean)*0.99),]
bin_data_df_mean = bin_data_df_mean.sort_values('Size').iloc[int(len(bin_data_df_mean)*0.01):int(len(bin_data_df_mean)*0.99),]

Fama_beta_Thu = pd.read_csv('E:/Stock_Data/Fama_beta_Thu.csv',index_col=0)
bin_data_Thu = pd.merge(Fama_beta_Thu,bin_data_df_mean,left_index=True,right_index=True,how='inner')

In [128]:
# bin_data_Thu.describe().to_csv('E:/Stock_Data/describe_data/bin_data_Thu_haveOutPoint.csv')
bin_data_Thu.describe().to_csv('E:/Stock_Data/describe_data/bin_data_Thu_noOutPoint.csv')

In [11]:
# 盈利能力分位数与负周四效应占比
stock_download_return_temp = os.listdir('E:/Stock_Data/stock_return_data/')
stock_download_return = [x[:-4] for x in stock_download_return_temp]
X = dummy_df.iloc[:1943,:]
rf = pd.read_csv('E:/Stock_Data/rf.csv',index_col=0)
for percent in [0.5,0.45,0.4,0.35,0.3,0.25,0.2,0.15,0.1,0.05]:
# for percent in [0.05]:
    cnt = 0
    tol_sig = 0
    pvalue_01 = {1:0,2:0,3:0,4:0,5:0}
    pvalue_pos = {1:0,2:0,3:0,4:0,5:0}
    Thu_record = []
    for code in bin_data_df_mean.sort_values('OP').iloc[-int(len(bin_data_df_mean)*percent):,].index:
        if code not in stock_download_return:
            continue
        y_temp = pd.read_csv('E:/Stock_Data/stock_return_data/'+code+'.csv',index_col = 0)
        if len(y_temp)<1943:
            continue   


        y = y_temp.iloc[:1943,:]

        # 用OLS
    #     X = sm.add_constant(X)
        # 是否使用超额收益率
        y = y - rf

        est = sm.OLS(y,X)
        result = est.fit()

        # 统计专用

#         # 显著结果统计(有显著性水平要求0.15)
#         for i in range(5):
#             if list(result.pvalues)[i]<0.15:
#                 pvalue_015[i+1] += 1 
        # 显著结果统计(有显著性水平要求0.1)
        for i in range(5):
            if list(result.pvalues)[i]<0.1:
                pvalue_01[i+1] += 1
        
        Thu_record.append(list(result.params)[3])
#         # 显著结果统计(有显著性水平要求0.05)
#         for i in range(5):
#             if list(result.pvalues)[i]<0.05:
#                 pvalue_005[i+1] += 1
#         # 显著结果统计(有显著性水平要求0.01)
#         for i in range(5):
#             if list(result.pvalues)[i]<0.01:
#                 pvalue_001[i+1] += 1

        # 显著结果中正负统计(有显著性水平要求0.1)
        for i in range(5):
            if list(result.pvalues)[i]<0.1 and list(result.params)[i]>0:
                pvalue_pos[i+1] += 1

        # 最显著结果统计(无显著性水平要求)
#         result_save_dct[np.argmin(list(result.pvalues))+1]+=1    

        # 正负统计-回归、其实就是样本均值
#         for i in range(5):
#             if list(result.params)[i]>0:
#                 positive[i+1] += 1

        # F统计量显著统计
#         if result.f_pvalue < 0.1:
#             f_pvalue += 1

        # 收集周四收益率数据
    #     y_Thu = y[X[4]==1]['return']
    #     if cnt == 0:
    #         y_Thu_save=y_Thu
    #     else:
    #         y_Thu_save=pd.concat([y_Thu_save,y_Thu],axis=0)

        # 收集周四平均收益率数据
    #     y_Thu = list(result.params)[3]
    #     Thu_mean_r.append(y_Thu)

    #     # 频率统计，周四为负
    #     y_Thu = y[X[4]==1]['return']
    #     y_Thu_std = y_Thu.std()
    #     Thu_neg += sum(y_Thu.apply(lambda x: 1 if x<-3*y_Thu_std else 0))
    #     Thu_pos += sum(y_Thu.apply(lambda x: 1 if x> 3*y_Thu_std else 0))
    #     # 频率统计，周一为正
    #     y_Mon = y[(X[2]==0) & (X[3]==0) & (X[4]==0) & (X[5]==0)]['return']
    #     y_Mon_std = y_Mon.std()
    #     Mon_pos += sum(y_Mon.apply(lambda x: 1 if x> 3*y_Mon_std else 0))
    #     Mon_neg += sum(y_Mon.apply(lambda x: 1 if x<-3*y_Mon_std else 0))
        # 频率统计，平均数正负

        # 频率统计，周四小于周一
    #     y_Mon = y[(X[2]==0) & (X[3]==0) & (X[4]==0) & (X[5]==0) ]['return']
    #     y_Thu = y[X[4]==1]['return']
    #     y_delta = y_Thu - y_Mon
    #     Thu_lower_Mon += sum(y_delta.apply(lambda x: 1 if x<0 else 0))  

        # 用arch
    #     reg = arch_model(y, x=X, mean='ARX', lags=0, vol='Garch', p=1, o=0, q=1, power=1, dist='ged', hold_back=None)
    #     result = reg.fit()
    #     result_save_dct[np.argmin(list(result.pvalues[:5]))+1]+=1

        cnt += 1
    print('显著结果统计0.1',pvalue_01,'\n显著结果中正负统计',pvalue_pos,'\n总数',cnt)
    print(pvalue_01[4]/cnt)

显著结果统计0.1 {1: 69, 2: 55, 3: 30, 4: 372, 5: 55} 
显著结果中正负统计 {1: 11, 2: 54, 3: 29, 4: 0, 5: 51} 
总数 709
0.5246826516220028
显著结果统计0.1 {1: 63, 2: 52, 3: 26, 4: 330, 5: 51} 
显著结果中正负统计 {1: 11, 2: 51, 3: 25, 4: 0, 5: 48} 
总数 639
0.5164319248826291
显著结果统计0.1 {1: 58, 2: 48, 3: 25, 4: 296, 5: 47} 
显著结果中正负统计 {1: 11, 2: 47, 3: 24, 4: 0, 5: 44} 
总数 567
0.5220458553791887
显著结果统计0.1 {1: 47, 2: 44, 3: 20, 4: 251, 5: 42} 
显著结果中正负统计 {1: 10, 2: 43, 3: 19, 4: 0, 5: 39} 
总数 493
0.5091277890466531
显著结果统计0.1 {1: 39, 2: 41, 3: 18, 4: 213, 5: 38} 
显著结果中正负统计 {1: 9, 2: 40, 3: 17, 4: 0, 5: 35} 
总数 422
0.504739336492891
显著结果统计0.1 {1: 35, 2: 36, 3: 18, 4: 173, 5: 34} 
显著结果中正负统计 {1: 8, 2: 35, 3: 17, 4: 0, 5: 31} 
总数 349
0.49570200573065903
显著结果统计0.1 {1: 32, 2: 29, 3: 15, 4: 136, 5: 28} 
显著结果中正负统计 {1: 6, 2: 28, 3: 14, 4: 0, 5: 25} 
总数 279
0.4874551971326165
显著结果统计0.1 {1: 27, 2: 22, 3: 11, 4: 100, 5: 25} 
显著结果中正负统计 {1: 6, 2: 21, 3: 11, 4: 0, 5: 24} 
总数 210
0.47619047619047616
显著结果统计0.1 {1: 21, 2: 15, 3: 7, 4: 57, 5: 17

In [129]:
from scipy import stats
stats.ttest_1samp(Thu_record, [0])

Ttest_1sampResult(statistic=array([-15.72527325]), pvalue=array([2.42425806e-24]))

In [23]:
# 用OLS
X = bin_data_Thu.iloc[:,-5:-1]
X = sm.add_constant(X)
X['Size_2'] = X['Size']*X['Size']
X['B_M_2'] = X['B_M']*X['B_M']
X['OP_2'] = X['OP']*X['OP']
X['Inv_2'] = X['Inv']*X['Inv']

y = bin_data_Thu.iloc[:,0]
est = sm.OLS(y,X)
result = est.fit()
print(result.params*10000)
print(result.pvalues*1000000)
print(-result.params[1]/(result.params[5]*2))
print(-result.params[2]/(result.params[6]*2))
print(-result.params[3]/(result.params[7]*2))
print(-result.params[4]/(result.params[8]*2))

const    -28.905116
Size       0.002091
B_M       -5.313841
OP        56.067386
Inv       -0.096094
Size_2     0.000002
B_M_2      6.387901
OP_2     -88.745770
Inv_2      0.180659
dtype: float64
const     2.527957e-100
Size       5.319239e+05
B_M        1.196578e+05
OP         1.393853e+01
Inv        9.530333e+05
Size_2     3.334193e+05
B_M_2      1.685975e+04
OP_2       3.121320e+04
Inv_2      7.302433e+05
dtype: float64
-459.82811193306577
0.4159301400927542
0.3158876512769173
0.2659546553885859


In [14]:
X.describe()

,const,Size,B_M,OP,Inv,Size_2,B_M_2,OP_2,Inv_2
count,1412.0,1412.000000,1412.000000,1412.000000,1412.000000,1.412000e+03,1412.000000,1412.000000,1.412000e+03
mean,1.0,132.814249,0.442958,0.107828,0.251709,5.782916e+04,0.251372,0.015740,2.228650e-01
std,0.0,200.544307,0.234945,0.064154,0.399525,2.744845e+05,0.298622,0.019931,1.241192e+00
min,1.0,17.134425,0.002488,0.011783,-0.092453,2.935885e+02,0.000006,0.000139,1.013142e-11
25%,1.0,44.983325,0.277237,0.060114,0.088993,2.023500e+03,0.076860,0.003614,8.040557e-03
50%,1.0,71.573525,0.390031,0.097873,0.162781,5.122770e+03,0.152124,0.009579,2.649766e-02
75%,1.0,132.146508,0.558306,0.141081,0.274511,1.746270e+04,0.311706,0.019904,7.535635e-02
max,1.0,2181.927325,2.202764,0.419125,4.251257,4.760807e+06,4.852167,0.175665,1.807318e+01


In [68]:
# 5因子回归 - 得系数，用以下一步回归 + 日历效应检验回归，得负周四效应（周四平均超额收益率）
rf = pd.read_csv('E:/Stock_Data/rf.csv',index_col=0)

code_lst = os.listdir('E:/Stock_Data/stock_return_data')
result_save_dct = {1:0,2:0,3:0,4:0,5:0}
cnt = 0

f_pvalue = 0
Thu_mean_r = []
for code_csv in code_lst:
    y_temp = pd.read_csv('E:/Stock_Data/stock_return_data/'+code_csv,index_col = 0)
    if len(y_temp)<1943:
        continue
    y = y_temp.iloc[:1943,:]
    X = pd.concat([MKTF,SMB,HML,CMA,RMW],axis=1)
    
    # 用OLS - 1
    X = sm.add_constant(X)
    # 是否使用超额收益率
    y = y - rf

    est = sm.OLS(y,X)
    result = est.fit()
    
    Const = list(result.params)[0]
    MKTF_beta = list(result.params)[1]
    SMB_beta = list(result.params)[2]
    HML_beta = list(result.params)[3]
    CMA_beta = list(result.params)[4]
    RMW_beta = list(result.params)[5]
    
    # 用OLS - 2
    # 是否使用超额收益率
    X = dummy_df.iloc[:1943,]

    est = sm.OLS(y,X)
    result = est.fit()
    
    Thu = list(result.params)[3]
    Thu_Pvalue = list(result.pvalues)[3]
    
    if cnt == 0:
        initial_df = pd.DataFrame(columns=['Code','Thu','Thu_Pvalue','C','MKTF_beta','SMB_beta','HML_beta','CMA_beta','RMW_beta'],index=range(9999))
        initial_df.iloc[cnt,:] = [code_csv[:-4],Thu,Thu_Pvalue,Const,MKTF_beta,SMB_beta,HML_beta,CMA_beta,RMW_beta]
    else:
        initial_df.iloc[cnt,:] = [code_csv[:-4],Thu,Thu_Pvalue,Const,MKTF_beta,SMB_beta,HML_beta,CMA_beta,RMW_beta]
    
    cnt += 1
initial_df = initial_df.dropna()
initial_df.to_csv('E:/Stock_Data/Fama_beta_Thu.csv',index=False)

KeyboardInterrupt: 

In [123]:
# 因子风险溢价描述性统计
X = pd.concat([MKTF,SMB,HML,CMA,RMW],axis=1)
X.describe().to_csv('E:/Stock_Data/describe_data/risk_premuim.csv')

In [118]:
initial_df = pd.read_csv('E:/Stock_Data/Fama_beta_Thu.csv')
initial_df['isThu'] = initial_df['Thu_Pvalue'].apply(lambda x: 1 if x<0.1 else 0)

In [76]:
y = initial_df[['Thu']]
y.index=initial_df['Code']
# X = initial_df[['MKTF_beta','SMB_beta','HML_beta','CMA_beta','RMW_beta']]

X = initial_df[['CMA_beta']]

X.index=initial_df['Code']
X = sm.add_constant(X)

# est = sm.Logit(y,X)
est = sm.OLS(y,X)
result = est.fit()

print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    Thu   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     5.939
Date:                Fri, 12 Apr 2019   Prob (F-statistic):             0.0149
Time:                        16:05:18   Log-Likelihood:                 10512.
No. Observations:                1934   AIC:                        -2.102e+04
Df Residuals:                    1932   BIC:                        -2.101e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0024   2.42e-05   -100.505      0.0

In [80]:
# 生成所有5因子敏感系数的组合 2^5-1=31种
import itertools
list1 = ['MKTF_beta','SMB_beta','HML_beta','CMA_beta','RMW_beta']
list2 = []
for i in range(1,len(list1)+1):
    iter_ = itertools.combinations(list1,i)
    if i ==1:
        iter_ = [[i[0]] for i in iter_]
    else:
        iter_ = [list(i) for i in iter_]
    list2.append(list(iter_))

[[['MKTF_beta'], ['SMB_beta'], ['HML_beta'], ['CMA_beta'], ['RMW_beta']], [['MKTF_beta', 'SMB_beta'], ['MKTF_beta', 'HML_beta'], ['MKTF_beta', 'CMA_beta'], ['MKTF_beta', 'RMW_beta'], ['SMB_beta', 'HML_beta'], ['SMB_beta', 'CMA_beta'], ['SMB_beta', 'RMW_beta'], ['HML_beta', 'CMA_beta'], ['HML_beta', 'RMW_beta'], ['CMA_beta', 'RMW_beta']], [['MKTF_beta', 'SMB_beta', 'HML_beta'], ['MKTF_beta', 'SMB_beta', 'CMA_beta'], ['MKTF_beta', 'SMB_beta', 'RMW_beta'], ['MKTF_beta', 'HML_beta', 'CMA_beta'], ['MKTF_beta', 'HML_beta', 'RMW_beta'], ['MKTF_beta', 'CMA_beta', 'RMW_beta'], ['SMB_beta', 'HML_beta', 'CMA_beta'], ['SMB_beta', 'HML_beta', 'RMW_beta'], ['SMB_beta', 'CMA_beta', 'RMW_beta'], ['HML_beta', 'CMA_beta', 'RMW_beta']], [['MKTF_beta', 'SMB_beta', 'HML_beta', 'CMA_beta'], ['MKTF_beta', 'SMB_beta', 'HML_beta', 'RMW_beta'], ['MKTF_beta', 'SMB_beta', 'CMA_beta', 'RMW_beta'], ['MKTF_beta', 'HML_beta', 'CMA_beta', 'RMW_beta'], ['SMB_beta', 'HML_beta', 'CMA_beta', 'RMW_beta']], [['MKTF_beta', '

In [130]:
# 自主采样测算敏感系数的回归系数稳定性
cnt = 0
y = initial_df[['isThu']]
y.index=initial_df['Code']
save_df = pd.DataFrame(columns=['MKTF_beta', 'SMB_beta', 'HML_beta', 'CMA_beta', 'RMW_beta'],index=range(31))
def isPos(x):
    if x>0:
        return 1
    else:
        return 0
for x_list_tol in list2:
    for x_list in x_list_tol:
        
        X = initial_df[x_list]

        X.index=initial_df['Code']
        X = sm.add_constant(X)
        
        est = sm.Logit(y,X)
        result = est.fit()
        
        for col in x_list:
            save_df.iloc[cnt,:][col]=isPos(result.params[col])
        
        cnt += 1

Optimization terminated successfully.
         Current function value: 0.685173
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684872
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684874
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684265
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.681659
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684476
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684609
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683242
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.681588
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683627
  

In [131]:
save_df.sum()

MKTF_beta     9.0
SMB_beta      8.0
HML_beta     12.0
CMA_beta     16.0
RMW_beta      0.0
dtype: float64

In [121]:
# 自主采样测算敏感系数的回归系数显著性
cnt = 0
y = initial_df[['isThu']]
y.index=initial_df['Code']
save_df = pd.DataFrame(columns=['MKTF_beta', 'SMB_beta', 'HML_beta', 'CMA_beta', 'RMW_beta'],index=range(31))
def issig(x):
    if x<0.01:
        return 1
    else:
        return 0
for x_list_tol in list2:
    for x_list in x_list_tol:
        
        X = initial_df[x_list]

        X.index=initial_df['Code']
        X = sm.add_constant(X)
        
        est = sm.Logit(y,X)
        result = est.fit()
        
        for col in x_list:
            save_df.iloc[cnt,:][col]=issig(result.pvalues[col])
        
        cnt += 1

Optimization terminated successfully.
         Current function value: 0.685173
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684872
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684874
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684265
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.681659
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684476
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684609
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683242
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.681588
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683627
  

In [122]:
save_df.sum()

MKTF_beta     0.0
SMB_beta      0.0
HML_beta      4.0
CMA_beta     13.0
RMW_beta     15.0
dtype: float64

In [129]:
# 五因子描述性统计
X.describe().to_csv('E:/Stock_Data/describe_data/risk_premium_params.csv')

In [348]:
# 5因子回归

rf = pd.read_csv('E:/Stock_Data/rf.csv',index_col=0)

bin1 = 'Size'
bin2s = ['B_M','OP','Inv']
# bin2s = ['B_M','OP','Inv','pe_ratio','industry']
for bin2 in bin2s:
    file_name = 'E:/Stock_Data/'+bin1+'_'+bin2+'.csv'
    return_df = pd.read_csv('E:/Stock_Data/'+bin1+'_'+bin2+'.csv',index_col=0)
    return_df = return_df.dropna(how="any",axis=0)
    bin1_list = [bin1+str(i+1) for i in range(5)]
    
    bin2_list = [bin2+str(i+1) for i in range(5)]
    bin1_bin2_save_df = pd.DataFrame(index=bin1_list,columns=bin2_list)
    
    row = 0
    for bin1_ in bin1_list:
        col = 0
        for bin2_ in bin2_list:
            bin1_bin2_ = bin1_+bin2_
            # y减去无风险收益率
            y_temp = return_df.iloc[:1943,:]
            y_temp_0 = y_temp[[bin1_bin2_]]
            y_temp_0.columns=['return']
            y = y_temp_0 - rf

#             # 用Garch
#             reg = arch_model(y, x=X, mean='HARX', lags=0, vol='Garch', p=1, o=0, q=1, power=1.0, dist='ged', hold_back=None)
#             result = reg.fit()
            
            # 用OLS
            X = pd.concat([MKTF,SMB,HML,CMA,RMW],axis=1)
            X = sm.add_constant(X)
            est = sm.OLS(y,X)
            result = est.fit()           
            
            save1 = result.rsquared_adj
            save2 = list(result.pvalues)[0]

            # 测试 - 结果矩阵形式保存
            bin1_bin2_save_df.iloc[row,col] = 'adjR2%.2f a_pvalue%.2f'%(save1*100,save2)
            
            col += 1
        row += 1
    # 保存
    bin1_bin2_save_df.to_csv('E:/Stock_Data/calander_effect_adj/'+bin1+'_'+bin2+'.csv')

In [347]:
# 5因子回归修正

# 引入日历效应哑变量 - 星期
sample_df = pd.read_csv('E:/Stock_Data/stock_return_data/001896.XSHE.csv',index_col=0)
trade_day_serises = list(sample_df.index)

trade_day_df = pd.DataFrame(trade_day_serises)
trade_day_df.columns = ['weekday']
trade_day_df['weekday'] = trade_day_df['weekday'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d").weekday()+1)

dummy_df = pd.get_dummies(trade_day_df['weekday'])
dummy_df = dummy_df[[4]]
dummy_df.index = sample_df.index
dummy_df = dummy_df.iloc[:1943,:]

rf = pd.read_csv('E:/Stock_Data/rf.csv',index_col=0)

bin1 = 'Size'
bin2s = ['B_M','OP','Inv']
# bin2s = ['B_M','OP','Inv','pe_ratio','industry']
for bin2 in bin2s:
    file_name = 'E:/Stock_Data/'+bin1+'_'+bin2+'.csv'
    return_df = pd.read_csv('E:/Stock_Data/'+bin1+'_'+bin2+'.csv',index_col=0)
    return_df = return_df.dropna(how="any",axis=0)
    bin1_list = [bin1+str(i+1) for i in range(5)]
    
    bin2_list = [bin2+str(i+1) for i in range(5)]
    bin1_bin2_save_df = pd.DataFrame(index=bin1_list,columns=bin2_list)
    
    row = 0
    for bin1_ in bin1_list:
        col = 0
        for bin2_ in bin2_list:
            bin1_bin2_ = bin1_+bin2_
            # y减去无风险收益率
            y_temp = return_df.iloc[:1943,:]
            y_temp_0 = y_temp[[bin1_bin2_]]
            y_temp_0.columns=['return']
            y = y_temp_0 - rf

#             # 用Garch
#             reg = arch_model(y, x=X, mean='HARX', lags=0, vol='Garch', p=1, o=0, q=1, power=1.0, dist='ged', hold_back=None)
#             result = reg.fit()
            
            # 用OLS
            X = pd.concat([MKTF,SMB,HML,CMA,RMW,dummy_df],axis=1)
            X = sm.add_constant(X)
            est = sm.OLS(y,X)
            result = est.fit()           
            
            save1 = result.rsquared_adj
            save2 = list(result.pvalues)[0]
            save3 = list(result.params)[6]
            save4 = list(result.pvalues)[6]

            # 测试 - 结果矩阵形式保存
            bin1_bin2_save_df.iloc[row,col] = 'adjR2%.2f a_pvalue%.2f Thu%.2f Thu_pvalue%.2f' % (save1*100,save2,save3*100,save4)
            
            col += 1
        row += 1
    # 保存
    bin1_bin2_save_df.to_csv('E:/Stock_Data/calander_effect_adj/'+bin1+'_'+bin2+'adj.csv')

In [341]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 return   R-squared:                       0.752
Model:                            OLS   Adj. R-squared:                  0.751
Method:                 Least Squares   F-statistic:                     978.6
Date:                Tue, 02 Apr 2019   Prob (F-statistic):               0.00
Time:                        00:00:22   Log-Likelihood:                 6556.3
No. Observations:                1943   AIC:                        -1.310e+04
Df Residuals:                    1936   BIC:                        -1.306e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4.806e-05      0.000     -0.228      0.8

In [344]:
result.pvalues

const    8.198361e-01
MKTF     0.000000e+00
SMB      1.309001e-56
HML      1.095246e-45
CMA      8.225096e-02
RMW      7.022682e-01
4        9.014270e-02
dtype: float64